In [124]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
f=open('drive/My Drive/train.txt','r')
x_train=[]
y_train=[]
for i in f:
    l=i.split(';')
    y_train.append(l[1].strip())
    x_train.append(l[0])
f=open('drive/My Drive/test.txt','r')
x_test=[]
y_test=[]
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
f=open('drive/My Drive/val.txt','r')
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

In [108]:
def clean_text(data):
    data=re.sub(r"(#[\d\w\.]+)", '', data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data

In [109]:
texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

In [110]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1
print('Number of unique words: {}'.format(len(index_of_words)))

Number of unique words: 17094


In [111]:
num_classes=6
embed_num_dims=300
max_seq_len=500
class_names=['anger','sadness','fear','joy','surprise','love']

In [89]:
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)

In [112]:
encoding={'anger':0,'sadness':1,'fear':2,'joy':3,'surprise':4,'love':5}
y_train=[encoding[x] for x in data_train.Emotion]
y_test=[encoding[x] for x in data_test.Emotion]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [113]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath) as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix

In [114]:
fname='embeddings/wiki-news-300d-1M.vec'
if not os.path.isfile(fname):
    print('Downloading word vectors...')
    urllib.request.urlretrieve('https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip','wiki-news-300d-1M.vec.zip')
    print('Unzipping...')
    with zipfile.ZipFile('wiki-news-300d-1M.vec.zip', 'r') as zip_ref:
        zip_ref.extractall('embeddings')
    print('done.')
    os.remove('wiki-news-300d-1M.vec.zip')
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)
embedd_matrix.shape

(17095, 300)

In [115]:
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
if bidirectional:
    model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
else:
    model.add(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          5128500   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               330240    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 5,460,282
Trainable params: 331,782
Non-trainable params: 5,128,500
_________________________________________________________________


In [119]:
batch_size=128
epochs=8
hist=model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

Epoch 1/8
125/125 [==============================] - 158s 1s/step - loss: 0.5494 - accuracy: 0.8122 - val_loss: 0.3850 - val_accuracy: 0.8752
Epoch 2/8
125/125 [==============================] - 155s 1s/step - loss: 0.3495 - accuracy: 0.8788 - val_loss: 0.2616 - val_accuracy: 0.9070
Epoch 3/8
125/125 [==============================] - 152s 1s/step - loss: 0.2627 - accuracy: 0.9044 - val_loss: 0.2197 - val_accuracy: 0.9180
Epoch 4/8
125/125 [==============================] - 158s 1s/step - loss: 0.2238 - accuracy: 0.9167 - val_loss: 0.1962 - val_accuracy: 0.9208
Epoch 5/8
125/125 [==============================] - 156s 1s/step - loss: 0.1983 - accuracy: 0.9209 - val_loss: 0.1837 - val_accuracy: 0.9277
Epoch 6/8
125/125 [==============================] - 164s 1s/step - loss: 0.1830 - accuracy: 0.9263 - val_loss: 0.1620 - val_accuracy: 0.9305
Epoch 7/8
125/125 [==============================] - 162s 1s/step - loss: 0.1707 - accuracy: 0.9293 - val_loss: 0.1695 - val_accuracy: 0.9320
Epoch 

In [122]:
message=['I am sad.']
seq=tokenizer.texts_to_sequences(message)
padded=pad_sequences(seq,maxlen=max_seq_len)
pred=model.predict(padded)
print('Message:'+str(message))
print('Emotion:',class_names[np.argmax(pred)])

Message:['I am sad.']
Emotion: sadness


In [125]:
tf.keras.models.save_model(model,'drive/My Drive/textmodel2',overwrite=True,include_optimizer=True,save_format=None,signatures=None,options=None)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: drive/My Drive/textmodel2/assets
